# 🚗 DCRNN Traffic Flow Prediction - Google Colab Edition

**Project**: Spatio-Temporal Traffic Flow Prediction  
**Author**: Vaishnavi Kamdi  
**Course**: Advanced ML, Fall 2025, GWU  

---

## 📋 What This Notebook Does

1. ✅ Clones your GitHub repository
2. ✅ Installs all dependencies
3. ✅ Enables GPU acceleration (T4/P100/V100)
4. ✅ Trains DCRNN model (10-30x faster than CPU)
5. ✅ Evaluates on test set
6. ✅ Generates visualizations
7. ✅ Downloads results to your local machine

---

## 🚀 Quick Start

**Before running**:
1. Go to `Runtime` → `Change runtime type` → Set `Hardware accelerator` to **GPU**
2. Click `Runtime` → `Run all` (or run cells sequentially)

**Expected time**: ~10-15 minutes on GPU (vs 2-4 hours on CPU!)

---

## 1️⃣ Setup: Clone Repository & Install Dependencies

In [ ]:
# Clone your GitHub repository (will get latest version with fixes)
!git clone https://github.com/vaish725/Spatio-Temporal-Traffic-Flow-Prediction.git
%cd Spatio-Temporal-Traffic-Flow-Prediction

# Ensure we have the latest code
!git pull origin main

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: No GPU detected. Go to Runtime → Change runtime type → Select GPU")

In [ ]:
# Install dependencies
!pip install -q torch-geometric
!pip install -q -r requirements.txt

print("✅ All dependencies installed!")

## 2️⃣ Verify Installation & Project Structure

In [ ]:
# Verify project structure
import os

required_files = [
    'models/dcrnn.py',
    'models/diffusion_conv.py',
    'src/dataset.py',
    'src/metrics.py',
    'scripts/train.py',
    'scripts/evaluate.py',
    'traffic_flow_setup.py'
]

print("📁 Checking project structure...")
all_exist = True
for file in required_files:
    exists = os.path.exists(file)
    status = "✅" if exists else "❌"
    print(f"{status} {file}")
    if not exists:
        all_exist = False

if all_exist:
    print("\n✅ All files present! Ready to train.")
else:
    print("\n❌ Some files missing. Check your repository.")

In [ ]:
# Test imports
print("🔍 Testing imports...")
try:
    from models.dcrnn import DCRNN
    from models.diffusion_conv import DiffusionConv
    from src.dataset import TrafficDataset, create_dataloaders
    from src.metrics import masked_mae, masked_rmse, masked_mape
    print("✅ All imports successful!")
except Exception as e:
    print(f"❌ Import error: {e}")

## 3️⃣ Train DCRNN Model (GPU-Accelerated)

This will train your model with:
- **GPU acceleration** (10-30x faster)
- **Early stopping** (patience=15)
- **Model checkpointing** (best & final models)
- **Progress tracking** with tqdm

**Expected time on GPU**: 10-20 minutes (vs 2-4 hours on CPU)

In [ ]:
# Train the model
!python3 scripts/train.py \
  --epochs 100 \
  --batch_size 64 \
  --hidden_dim 64 \
  --num_layers 2 \
  --lr 0.001 \
  --patience 15 \
  --checkpoint_dir checkpoints \
  --device cuda

### 📊 Training Summary

In [ ]:
# Display training history
import json
import matplotlib.pyplot as plt

with open('checkpoints/training_history.json', 'r') as f:
    history = json.load(f)

print("🎯 Training Summary")
print("=" * 50)
print(f"Total epochs trained: {len(history)}")
print(f"Best validation loss: {min([h['val_loss'] for h in history]):.4f}")
print(f"Best epoch: {min(history, key=lambda x: x['val_loss'])['epoch']}")
print(f"Training time: {sum([h['epoch_time'] for h in history]) / 60:.2f} minutes")

# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss curves
axes[0].plot([h['epoch'] for h in history], [h['train_loss'] for h in history], label='Train Loss', marker='o')
axes[0].plot([h['epoch'] for h in history], [h['val_loss'] for h in history], label='Val Loss', marker='s')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training & Validation Loss')
axes[0].legend()
axes[0].grid(True)

# MAE curves
axes[1].plot([h['epoch'] for h in history], [h['val_mae'] for h in history], label='Val MAE', marker='o', color='green')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].set_title('Validation MAE')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Training curves saved as 'training_curves.png'")

## 4️⃣ Evaluate Model on Test Set

In [ ]:
# Evaluate the model
!python3 scripts/evaluate.py \
  --checkpoint checkpoints/best_model.pt \
  --hidden_dim 64 \
  --num_layers 2 \
  --plot \
  --save_predictions \
  --device cuda

### 📈 Evaluation Results

In [ ]:
# Display evaluation metrics
import json

with open('results/metrics.json', 'r') as f:
    metrics = json.load(f)

print("🎯 Test Set Performance")
print("=" * 60)
print(f"Overall MAE:  {metrics['overall']['mae']:.4f}")
print(f"Overall RMSE: {metrics['overall']['rmse']:.4f}")
print(f"Overall MAPE: {metrics['overall']['mape']:.2f}%")
print()
print("Multi-Horizon Performance:")
print("-" * 60)
for horizon, vals in metrics['horizons'].items():
    print(f"{horizon:12s} → MAE: {vals['mae']:.4f}, RMSE: {vals['rmse']:.4f}, MAPE: {vals['mape']:.2f}%")

### 📊 Visualizations

In [ ]:
# Display generated plots
from IPython.display import Image, display

print("📈 Sample Predictions:")
display(Image('results/predictions.png'))

print("\n📊 Horizon-wise Performance:")
display(Image('results/horizon_metrics.png'))

print("\n📈 Training Curves:")
display(Image('training_curves.png'))

## 5️⃣ Download Results to Local Machine

This will create a ZIP file with all your results that you can download.

In [ ]:
# Create ZIP with all results
import shutil
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
zip_name = f'dcrnn_results_{timestamp}'

# Create results archive
!mkdir -p results_archive
!cp -r checkpoints results_archive/
!cp -r results results_archive/
!cp training_curves.png results_archive/

# Create ZIP
shutil.make_archive(zip_name, 'zip', 'results_archive')

print(f"✅ Results packaged as: {zip_name}.zip")
print(f"📦 Size: {os.path.getsize(f'{zip_name}.zip') / 1e6:.2f} MB")

In [ ]:
# Download the ZIP file
from google.colab import files

print("📥 Downloading results...")
files.download(f'{zip_name}.zip')
print("✅ Download complete! Check your browser's downloads folder.")

## 🔧 Optional: Advanced Configurations

Try different hyperparameters for better performance!

### Configuration 1: Larger Model

In [ ]:
# Train with larger hidden dimension
!python3 scripts/train.py \
  --epochs 100 \
  --batch_size 32 \
  --hidden_dim 128 \
  --num_layers 3 \
  --lr 0.001 \
  --patience 15 \
  --checkpoint_dir checkpoints_large \
  --device cuda

### Configuration 2: Learning Rate Decay

In [ ]:
# Train with learning rate decay
!python3 scripts/train.py \
  --epochs 100 \
  --batch_size 64 \
  --hidden_dim 64 \
  --num_layers 2 \
  --lr 0.001 \
  --lr_decay \
  --lr_decay_rate 0.5 \
  --patience 20 \
  --checkpoint_dir checkpoints_lr_decay \
  --device cuda

### Configuration 3: More Diffusion Hops

In [ ]:
# Train with more diffusion steps
!python3 scripts/train.py \
  --epochs 100 \
  --batch_size 64 \
  --hidden_dim 64 \
  --num_layers 2 \
  --max_diffusion_step 3 \
  --lr 0.001 \
  --patience 15 \
  --checkpoint_dir checkpoints_diffusion3 \
  --device cuda

---

## 📝 Notes

### GPU vs CPU Performance
- **CPU (local)**: 2-4 hours for 100 epochs
- **GPU (Colab)**: 10-20 minutes for 100 epochs
- **Speedup**: ~10-30x faster!

### Colab GPU Options
- **Free tier**: T4 GPU (16GB) - plenty for this project
- **Colab Pro**: V100/A100 (even faster)

### Tips
1. **Save frequently**: Colab sessions disconnect after 12 hours
2. **Mount Google Drive**: Store results automatically
3. **Use checkpoints**: Resume training if disconnected
4. **Download results**: Don't lose your trained models!

### Troubleshooting
- **No GPU**: Go to `Runtime` → `Change runtime type` → Select `GPU`
- **Disconnected**: Re-run all cells from the top
- **Out of memory**: Reduce `batch_size` or `hidden_dim`
- **Import errors**: Make sure repository is cloned correctly

---

## 🎉 Done!

Your DCRNN model is now trained on GPU! 🚀

**What's in your ZIP file**:
- `checkpoints/` - Trained models (best & final)
- `results/` - Metrics and visualizations
- `training_curves.png` - Training history plot

**Next steps**:
1. Extract the ZIP file on your local machine
2. Use results in your report/presentation
3. Compare with your CPU training results

---